***

**DUISC | SCEN | PMP | Scientific Computing and Data Analysis<br>
P_ISC35 | Python | ODE-1 | worksheet 2 | solving first-order ODEs | numerical solution using the Euler method | implementation in C<br>
Jeff Davidson | 2024-01-29**

***

# Solving a first-order ODE using the Euler method
The Euler method is one of the simplest numerical methods for solving ordinary differential equations (ODEs). The _forward_ Euler method can be used to approximate the derivative by considering small increments in time as shown in $(1)$. Typically, a fixed _step size_ $h$ is used, resulting in $(2)$. Rearranging $(2)$, we can obtain an equation to calculate the _next_ value base on the _current_ value and the gradient at the current point in time, such as $(3)$. The gradient $\dfrac{dy}{dt}$ would be determined by some function such as $f(y_i,t_i)$ and may also be based on other variables such as inputs to a system, resulting in the general form of $(4)$.

\begin{align}
    \dfrac{dy}{dt} &\approx \dfrac{y_{i+1}-y_{i}}{t_{i+1}-t_{i}} \tag{1}\\
    \dot{y} &\approx \dfrac{y_{i+1}-y_{i}}{h} \tag{2}\\
    y_{i+1} &= y_{i} + h\dot{y} \tag{3}\\
    y_{i+1} &= y_{i} + hf(y_i,t_i) \tag{4}\\
    \text{next} &= \text{current} + (\text{step}\times\text{gradient})
\end{align}

## The R-C circuit example
As described in the previous worksheet, the ODE for our example RC system is shown in Equation $(5)$.
\begin{align}
\dot{y} &= \dfrac{x(t)-y(t)}{\tau}\tag{5}
\end{align}

### Euler numercial solution
For our system, we can see that the derivative or gradient depends on the values of both our input $x$ and output $y$, both of which can change over time. For systems were our derivatives are with respect to time, Newton's shorter _dot_ notation is commonly used. Referring back to our general Euler equations, we can now write an equation to compute the _next_ value of $y$ based on _current_ values of both $x$ and $y$. 

\begin{align}
y_{i+1} &= y_{i} + h \dot{y} \tag{6}\\
\dot{y} &= f(x_i, y_i)  \tag{7}
\end{align}

To implement this solution in code, we simply need to create and populate vectors for $t$ and $x$, create an ODE function $\dot{y}$, set an initial value for the solution variable $y_{0}$ and then use a basic loop to iterate over all values in $t$, calculating values of $y_{i+1}$ where $i=0,1,2,\ldots$ using the $\dot{y}$ ODE function.

We an make use of the `vector` data type in `C++`. Online documentation can be found here: [std::vector](https://en.cppreference.com/w/cpp/container/vector)

In [4]:
#include <iostream>
#include <fstream>
#include <string>
#include <vector>

using namespace std;

### System structure and general functions

#### Structure to hold system parameters and data vectors

In [5]:
struct aSystem
{
    // properties of the body
    string title;      // title: used on graphs
    double g;          // acceleration due to gravity
    double m;          // mass of falling object(kg)
    double k_d;        // combined drag coefficient
    
    // vector type arrays to hold the computed data points
    vector<double> t = {};    // time (s)
    vector<double> x = {};    // distance x(t)
    vector<double> v = {};    // velocity v(t)
}

#### General functions to display data and save as CSV file

In [6]:
void system_dump(aSystem *ptr)
{
    int i;
    
    cout << "title = " << ptr->title << endl;
    cout << scientific << setprecision(6);
    cout << "g = " << ptr->g << endl;
    cout << "m = " << ptr->m << endl;
    cout << "k_d = " << ptr->k_d << endl;
    cout << "size(t) = " << ptr->t.size() << endl;
    cout << "size(x) = " << ptr->x.size() << endl;
    cout << "size(v) = " << ptr->v.size() << endl;

    // header
    cout << "i, t, x, v" << endl;

    // data as CSV
    if ((ptr->t.size() > 0) && (ptr->x.size() > 0) && (ptr->v.size() > 0))
    {
        for (i=0; i<ptr->t.size(); i++)
        {
            cout << i << ", ";
            cout << ptr->t[i] << ", ";
            cout << ptr->x[i] << ", ";
            cout << ptr->v[i] << endl;
        }
    }
    else
    {
        cout << "no data" << endl;
    }
}

input_line_14:5:41: error: reference to overloaded function could not be resolved; did you mean to call it?
    cout << "title = " << ptr->title << endl;
                                        ^~~~
/../lib/gcc/aarch64-linux-gnu/11/../../../../include/c++/11/ostream:684:5: note: possible target for call
    endl(basic_ostream<_CharT, _Traits>& __os)
    ^
/../lib/gcc/aarch64-linux-gnu/11/../../../../include/c++/11/ostream:245:7: note: candidate function not viable: no overload of 'endl' matching 'const void *' for
      1st argument
      operator<<(const void* __p)
      ^
/../lib/gcc/aarch64-linux-gnu/11/../../../../include/c++/11/string_view:667:5: note: candidate function [with _CharT = char, _Traits = std::char_traits<char>] not
      viable: no overload of 'endl' matching 'basic_string_view<char,
      std::char_traits<char> >' for 2nd argument
    operator<<(basic_ostream<_CharT, _Traits>& __os,
    ^
/../lib/gcc/aarch64-linux-gnu/11/../../../../include/c++/11/system_error:279:5

In [ ]:
void write_parameters_csv(aSystem *ptr, string csv_path)
{
    // write parameters to CSV file
    ofstream csv_file;

    csv_file.open(csv_path);

    // header
    csv_file << "parameter,value" << endl;

    // data
    csv_file << "title," << ptr->title << endl;
    csv_file << scientific << setprecision(6);
    csv_file << "g," << ptr->g << endl;
    csv_file << "m," << ptr->m << endl;
    csv_file << "k_d," << ptr->k_d << endl;

    csv_file.close();
}

In [ ]:
void write_data_csv(aSystem *ptr, string csv_path)
{
    // write data to CSV file
    ofstream csv_file;
    int i;

    csv_file.open(csv_path);

    // header
    csv_file << "i,t,x,v" << endl;

    // data
    if ((ptr->t.size() > 0) && (ptr->x.size() > 0) && (ptr->v.size() > 0))
    {
        for (i=0; i<ptr->t.size(); i++)
        {
            csv_file << i << ",";
            csv_file << scientific << setprecision(6);
            csv_file << ptr->t[i] << ",";
            csv_file << ptr->x[i] << ",";
            csv_file << ptr->v[i] << endl;
        }
    }

    csv_file.close();
}

#### The system ODE function

In [ ]:
double vdot(double *curr_state, aSystem *the_system)
{   
    double x = curr_state[0];
    double v = curr_state[1];
    double g = the_system->g;
    double m = the_system->m;
    double k_d = the_system->k_d;
    
    return g - ((k_d * v * fabs(v)) / m);
}

In [ ]:
double xdot(double *curr_state, aSystem *the_system)
{
    double x = curr_state[0];
    double v = curr_state[1];

    return v;
}

#### The ODE solver function using the Euler method

In [ ]:
int ode_solver_euler(
double (*f_ode_1)(double*, aSystem*),
double (*f_ode_2)(double*, aSystem*),
double *initial_conditions,
aSystem *the_system
)
{
    // local variables
    int i;
    double h;
    double curr_state[2];
    double v0;
    doubel x0;

    // we will write solution data directly into system structure
    
    // append initial values
    x0 = initial_conditions[0];
    v0 = initial_conditions[0];
    the_system->x.push_back(x0);
    the_system->v.push_back(v0);
    
    // computational loop
    for (i=0; i < the_system->t.size()-1; i++)
    {
        // calcuate step size from t vector
        h = the_system->t[i+1] - the_system->t[i];

        // current states (input x, and ouput y)
        curr_state[0] = the_system->x[i];
        curr_state[1] = the_system->v[i];
        
        // Euler method to compute next value of solution
        // y[i+1] = y[i] + (h * ydot(x[i], y[i]))
        the_system->v.push_back(the_system->v[i] + (h * f_ode_1(curr_state, the_system)));
        the_system->x.push_back(the_system->x[i] + (h * f_ode_2(curr_state, the_system)));
    }
    
    // return the number of computations
    return i+1;
}

### Main program

#### Create a test system for the RC circuit

In [ ]:
// MAIN PROGRAM

// create a system for the RC circuit
// we just need to pass the values of the first three variables in the structure
struct aSystem falling_jim = {"Falling jim", 9.81, 67, 0.24};

#### Create vectors for time $t$ and input $x$

In [ ]:
// create a vector of time values in our system
int i = 0;
int n = 101;
double T = 50;
double h = T / (n - 1);

for (i=0; i<n; i++)
{
    falling_jim.t.push_back(h * i);
}

system_dump(&falling_jim);

#### Compute solution data for output $y$

In [ ]:
// create solution data for x and v
int n = 0;
double initial_conditions[2] = (0, 0);

n = ode_solver_euler(&vdot, &xdot, initial_conditions, &falling_jim);

system_dump(&falling_jim);

#### Write parameters and data to CSV files

In [ ]:
// write CSV files
cout << "writing CSV files ... ";
write_parameters_csv(&falling_jim, "parameters.csv");
write_data_csv(&falling_jim, "data_cxx.csv");
cout << "done" << endl;

In [ ]:
%%timeit
ode_solver_euler(&vdot, &xdot, initial_conditions, &falling_jim);